In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Dataframe for Evergreen Database
Evgn = pd.read_csv("EvergreenDatasetRaw_Dataset_2005-2018_v02.csv")
#Dataframe for individual drugs from Orange Book
Product = pd.read_csv('products.txt',delimiter="~")
#Dataframe for individual patents from Orange Book
Patent = pd.read_csv("patent.txt",delimiter="~")
#Dataframe for linking NDA (New Drug Application Number) to NDC (National Drug Code)
Link = pd.read_csv("20220906_product.csv")
#Dataframe for 

In [4]:
Patent['EvergreenFlag'] = [0] * len(Patent)
Patent.loc[Patent['Patent_No'].isin(Evgn['Patent Number']),'EvergreenFlag'] = 1

In [5]:
print(len(Patent))
Patent['EvergreenFlag'].value_counts(normalize=True)

17835


1    0.621082
0    0.378918
Name: EvergreenFlag, dtype: float64